In [1]:
%scala
import java.net.URL
import java.io.File
import org.apache.commons.io.FileUtils
import org.apache.spark.sql.functions._
import scala.util.Try

dbutils.fs.unmount("/mnt/test")
dbutils.fs.mount(
  source = "wasbs://mag-2019-01-25@iunimag.blob.core.windows.net/",
  mountPoint = "/mnt/test",
  extraConfigs = Map("fs.azure.account.key.iunimag.blob.core.windows.net" -> dbutils.secrets.get(scope = "AzureKeyVaultTest", key = "blobKeyDatabricks")))


//val df = spark.read.parquet("adl://iunimag.azuredatalakestore.net/WoS/CORE_2014_2015_Annuals")
display(dbutils.fs.ls("/mnt/test"))

path,name,size
dbfs:/mnt/test/AIpapersAll.tsv,AIpapersAll.tsv,933089982
dbfs:/mnt/test/AIpapersCitationsAll.tsv,AIpapersCitationsAll.tsv,8090967470
dbfs:/mnt/test/AIpapersExtend.tsv,AIpapersExtend.tsv,7752154999
dbfs:/mnt/test/AIpapersOthers.tsv,AIpapersOthers.tsv,7402542807
dbfs:/mnt/test/AIpapersPatent.tsv,AIpapersPatent.tsv,157413931
dbfs:/mnt/test/AIpapersWoS.csv.gz,AIpapersWoS.csv.gz,301064055
dbfs:/mnt/test/KatzData2.tar,KatzData2.tar,443852800
dbfs:/mnt/test/MAGwosJournalMatch/,MAGwosJournalMatch/,0
dbfs:/mnt/test/PMwosCitations.csv.gz,PMwosCitations.csv.gz,3186720390
dbfs:/mnt/test/YangyangExtra.csv.gz,YangyangExtra.csv.gz,1633659711


In [2]:
import pyspark.sql.functions as f
WoSjMap = spark.read.format("parquet").load("/mnt/test/MAGwosJournalMatch/*.parquet").toDF("MAGID","doi2","MJid", "WoSID", "WoSyear", "WoSjournal")
WoSjMap.describe().filter(f.col("summary") == "count").show()

+-------+--------+--------+--------+--------+--------+----------+
summary| MAGID| doi2| MJid| WoSID| WoSyear|WoSjournal|
+-------+--------+--------+--------+--------+--------+----------+
 count|18988668|18988668|17874374|18988668|18988668| 18988668|
+-------+--------+--------+--------+--------+--------+----------+

In [3]:
from pyspark.sql.functions import col
WoSjMap.dropDuplicates().describe().filter(f.col("summary") == "count").show()
WoSjMap.groupBy("MAGID").count().sort(col("count").desc()).show()

+-------+--------+--------+--------+--------+--------+----------+
summary| MAGID| doi2| MJid| WoSID| WoSyear|WoSjournal|
+-------+--------+--------+--------+--------+--------+----------+
 count|18988668|18988668|17874374|18988668|18988668| 18988668|
+-------+--------+--------+--------+--------+--------+----------+

+----------+-----+
 MAGID|count|
+----------+-----+
2565706482| 75|
2547451769| 75|
1575564593| 75|
1488671686| 75|
2573844077| 70|
2519742608| 63|
2302468765| 63|
2616032266| 63|
2890385727| 63|
 614392456| 63|
2588274710| 63|
2592300128| 63|
 583572480| 37|
 4683573| 34|
1269725168| 34|
 98803720| 34|
 66859782| 29|
2260556489| 25|
1523071718| 24|
2811467015| 23|
+----------+-----+
only showing top 20 rows

In [4]:
openPapers = spark.read.format("csv").option("header", "true").option("escape", "\"").option("multiLine", "true").load("/mnt/test/MAGwosJournalMatch/OpenSci3.csv")
openPapers.describe().filter(f.col("summary") == "count").show()

+-------+----+-------+--------------+----------------+-----------+------------+-----------+----------+----+----+----+----+----+----+--------+----+-------+-------+---------+----+-----+-----------------+---------------+
summary| Tag|PaperId|AuthorIdsOrder|AuthorNamesOrder|authorCount|unknownCount|femaleCount|femaleProp| 1st| 2nd| 3rd| 4th| 5th|last|FoSNames|Year|DocType|Journal|Publisher| Doi|Title|EstimatedCitation|IndexedAbstract|
+-------+----+-------+--------------+----------------+-----------+------------+-----------+----------+----+----+----+----+----+----+--------+----+-------+-------+---------+----+-----+-----------------+---------------+
 count|2945| 2945| 2945| 2945| 2945| 2945| 2945| 2945|2945|2945|2945|2945|2945|2945| 2945|2945| 2945| 2945| 2945|2945| 2945| 2945| 2945|
+-------+----+-------+--------------+----------------+-----------+------------+-----------+----------+----+----+----+----+----+----+--------+----+-------+-------+---------+----+-----+-----------------+---------------+

In [5]:
from pyspark.sql.functions import countDistinct

openPapersJ = openPapers.join(WoSjMap, (openPapers.PaperId == WoSjMap.MAGID), "left")
openPapersJ.describe().filter(f.col("summary") == "count").show()
openPapersJ0 = openPapersJ.groupBy("PaperId").count().sort(col("count").desc()).show()

+-------+----+-------+--------------+----------------+-----------+------------+-----------+----------+----+----+----+----+----+----+--------+----+-------+-------+---------+----+-----+-----------------+---------------+-----+----+----+-----+-------+----------+
summary| Tag|PaperId|AuthorIdsOrder|AuthorNamesOrder|authorCount|unknownCount|femaleCount|femaleProp| 1st| 2nd| 3rd| 4th| 5th|last|FoSNames|Year|DocType|Journal|Publisher| Doi|Title|EstimatedCitation|IndexedAbstract|MAGID|doi2|MJid|WoSID|WoSyear|WoSjournal|
+-------+----+-------+--------------+----------------+-----------+------------+-----------+----------+----+----+----+----+----+----+--------+----+-------+-------+---------+----+-----+-----------------+---------------+-----+----+----+-----+-------+----------+
 count|2946| 2946| 2946| 2946| 2946| 2946| 2946| 2946|2946|2946|2946|2946|2946|2946| 2946|2946| 2946| 2946| 2946|2946| 2946| 2946| 2946| 1500|1500|1411| 1500| 1500| 1500|
+-------+----+-------+--------------+----------------+-----------+------------+-----------+----------+----+----+----+----+----+----+--------+----+-------+-------+---------+----+-----+-----------------+---------------+-----+----+----+-----+-------+----------+

+----------+-----+
 PaperId|count|
+----------+-----+
1995609007| 2|
2000157697| 1|
1993144913| 1|
2044576353| 1|
2085022762| 1|
2113984786| 1|
2149697218| 1|
2156608586| 1|
2178250534| 1|
2253112347| 1|
2312788655| 1|
2519957376| 1|
2593267026| 1|
2598956343| 1|
2751429499| 1|
2771769497| 1|
2781115165| 1|
1572221326| 1|
1963989273| 1|
1997015862| 1|
+----------+-----+
only showing top 20 rows

In [6]:
openPapersJ.filter(f.col("PaperId") == 1995609007).show(20)
openPapersJ.filter(f.col("WoSID") == "WOS:000348245400019").show(20)

+---------------+----------+--------------+------------------+-----------+------------+-----------+----------+---+---+---+---+---+----+--------------------+----+-------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+----------+--------------------+--------+-------------------+-------+--------------------+
 Tag| PaperId|AuthorIdsOrder| AuthorNamesOrder|authorCount|unknownCount|femaleCount|femaleProp|1st|2nd|3rd|4th|5th|last| FoSNames|Year|DocType| Journal| Publisher| Doi| Title|EstimatedCitation| IndexedAbstract| MAGID| doi2| MJid| WoSID|WoSyear| WoSjournal|
+---------------+----------+--------------+------------------+-----------+------------+-----------+----------+---+---+---+---+---+----+--------------------+----+-------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+----------+--------------------+--------+-------------------+-------+--------------------+
Reproducibility|1995609007| 2481659591|Charles W. Schmidt| 1| 0| 0| 0| 0| NA| NA| NA| NA| 0|medicine; clinica...|2014|Journal|Environmental Hea...|National Institut...|10.1289/ehp.122-A188|Research wrangler...| 6|Advances in scien...|1995609007|10.1289/ehp.122-A188|10446503|WOS:000338928000007| 2014|ENVIRONMENTAL HEA...|
Reproducibility|1995609007| 2481659591|Charles W. Schmidt| 1| 0| 0| 0| 0| NA| NA| NA| NA| 0|medicine; clinica...|2014|Journal|Environmental Hea...|National Institut...|10.1289/ehp.122-A188|Research wrangler...| 6|Advances in scien...|1995609007|10.1289/ehp.122-A188|10446503|WOS:000348245400019| 2014|SALUD PUBLICA DE ...|
+---------------+----------+--------------+------------------+-----------+------------+-----------+----------+---+---+---+---+---+----+--------------------+----+-------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+----------+--------------------+--------+-------------------+-------+--------------------+

+---------------+----------+--------------+------------------+-----------+------------+-----------+----------+---+---+---+---+---+----+--------------------+----+-------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+----------+--------------------+--------+-------------------+-------+--------------------+
 Tag| PaperId|AuthorIdsOrder| AuthorNamesOrder|authorCount|unknownCount|femaleCount|femaleProp|1st|2nd|3rd|4th|5th|last| FoSNames|Year|DocType| Journal| Publisher| Doi| Title|EstimatedCitation| IndexedAbstract| MAGID| doi2| MJid| WoSID|WoSyear| WoSjournal|
+---------------+----------+--------------+------------------+-----------+------------+-----------+----------+---+---+---+---+---+----+--------------------+----+-------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+----------+--------------------+--------+-------------------+-------+--------------------+
Reproducibility|1995609007| 2481659591|Charles W. Schmidt| 1| 0| 0| 0| 0| NA| NA| NA| NA| 0|medicine; clinica...|2014|Journal|Environmental Hea...|National Institut...|10.1289/ehp.122-A188|Research wrangler...| 6|Advances in scien...|1995609007|10.1289/ehp.122-A188|10446503|WOS:000348245400019| 2014|SALUD PUBLICA DE ...|
+---------------+----------+--------------+------------------+-----------+------------+-----------+----------+---+---+---+---+---+----+--------------------+----+-------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+----------+--------------------+--------+-------------------+-------+--------------------+

In [7]:
from pyspark.sql.functions import countDistinct
openPapersJ2 = openPapers.join(WoSjMap, openPapers.Doi == WoSjMap.doi2, "left")
openPapersJ2.describe().filter(f.col("summary") == "count").show()

+-------+----+-------+--------------+----------------+-----------+------------+-----------+----------+----+----+----+----+----+----+--------+----+-------+-------+---------+----+-----+-----------------+---------------+-----+----+----+-----+-------+----------+
summary| Tag|PaperId|AuthorIdsOrder|AuthorNamesOrder|authorCount|unknownCount|femaleCount|femaleProp| 1st| 2nd| 3rd| 4th| 5th|last|FoSNames|Year|DocType|Journal|Publisher| Doi|Title|EstimatedCitation|IndexedAbstract|MAGID|doi2|MJid|WoSID|WoSyear|WoSjournal|
+-------+----+-------+--------------+----------------+-----------+------------+-----------+----------+----+----+----+----+----+----+--------+----+-------+-------+---------+----+-----+-----------------+---------------+-----+----+----+-----+-------+----------+
 count|2976| 2976| 2976| 2976| 2976| 2976| 2976| 2976|2976|2976|2976|2976|2976|2976| 2976|2976| 2976| 2976| 2976|2976| 2976| 2976| 2976| 1527|1527|1435| 1527| 1527| 1527|
+-------+----+-------+--------------+----------------+-----------+------------+-----------+----------+----+----+----+----+----+----+--------+----+-------+-------+---------+----+-----+-----------------+---------------+-----+----+----+-----+-------+----------+

In [8]:
openPapersJfinal = openPapersJ.filter((f.col("WoSID") != "WOS:000348245400019")|f.col("WoSID").isNull()).drop("MAGID", "PaperId", "doi2", "WoSyear", "MJid")
openPapersJfinal.describe().filter(f.col("summary") == "count").show()
openPapersJfinal.show()

+-------+----+--------------+----------------+-----------+------------+-----------+----------+----+----+----+----+----+----+--------+----+-------+-------+---------+----+-----+-----------------+---------------+-----+----------+
summary| Tag|AuthorIdsOrder|AuthorNamesOrder|authorCount|unknownCount|femaleCount|femaleProp| 1st| 2nd| 3rd| 4th| 5th|last|FoSNames|Year|DocType|Journal|Publisher| Doi|Title|EstimatedCitation|IndexedAbstract|WoSID|WoSjournal|
+-------+----+--------------+----------------+-----------+------------+-----------+----------+----+----+----+----+----+----+--------+----+-------+-------+---------+----+-----+-----------------+---------------+-----+----------+
 count|2945| 2945| 2945| 2945| 2945| 2945| 2945|2945|2945|2945|2945|2945|2945| 2945|2945| 2945| 2945| 2945|2945| 2945| 2945| 2945| 1499| 1499|
+-------+----+--------------+----------------+-----------+------------+-----------+----------+----+----+----+----+----+----+--------+----+-------+-------+---------+----+-----+-----------------+---------------+-----+----------+

+---------------+--------------------+--------------------+-----------+------------+-----------+-------------------+---+---+---+---+---+----+--------------------+----+----------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+-------------------+--------------------+
 Tag| AuthorIdsOrder| AuthorNamesOrder|authorCount|unknownCount|femaleCount| femaleProp|1st|2nd|3rd|4th|5th|last| FoSNames|Year| DocType| Journal| Publisher| Doi| Title|EstimatedCitation| IndexedAbstract| WoSID| WoSjournal|
+---------------+--------------------+--------------------+-----------+------------+-----------+-------------------+---+---+---+---+---+----+--------------------+----+----------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+-------------------+--------------------+
Reproducibility|2481862173; 26782...|P Kukolowicz; W B...| 3| 3| 0| NA| NA| NA| NA| NA| NA| NA|nuclear medicine;...|2015| Journal| Medical Physics|American Associat...| 10.1118/1.4925040|SU-E-T-677: Repro...| 0|Purpose: To compa...| null| null|
Reproducibility|2132317228; 24369...|Marcio De Menezes...| 4| 1| 0| 0| 0| 0| 0| NA| NA| NA|systematic error;...|2010| Journal|Journal of Oral a...| Elsevier|10.1016/j.joms.20...|Accuracy and Repr...| 83|Purpose To test t...|WOS:000281835400016|JOURNAL OF ORAL A...|
Reproducibility|2004717454; 24290...|Etienne Cavaignac...| 8| 5| 0| 0| 0| NA| NA| NA| 0| 0|reproducibility; ...|2013| Journal|Orthopaedics & Tr...|Orthop Traumatol ...|10.1016/j.otsr.20...|CT scan does not ...| 8|Summary Introduct...|WOS:000314635900007|ORTHOPAEDICS & TR...|
 OpenScience| 2081393523| David Tyfield| 1| 0| 0| 0| 0| NA| NA| NA| NA| 0|open science; soc...|2013| Journal|Spontaneous Gener...| NA|10.4245/sponge.v7...|Transition to Sci...| 0|The present is a ...| null| null|
Reproducibility| 2151748632| Ryan A. Harris| 1| 0| 0| 0| 0| NA| NA| NA| NA| 0|physical therapy;...|2010| Journal|European Journal ...| Springer-Verlag|10.1007/s00421-00...|FMD, reproducibil...| 1| NA|WOS:000277177000023|EUROPEAN JOURNAL ...|
 OpenScience|2748670092; 26487...|Maryann Martone; ...| 12| 0| 3| 0.25| 1| 1| 0| 0| 0| 0|open science; dat...|2016| Journal|Research Ideas an...| Pensoft Publishers| 10.3897/rio.2.e8424|ContentMine/Hypot...| 1| NA| null| null|
Reproducibility|2154509907; 20732...|Kenji Kashima; Da...| 2| 0| 0| 0| 0| 0| NA| NA| NA| 0|computer science;...|2014| Journal|Journal of the So...| NA|10.9746/sicetr.50...|Simultaneous Use ...| 0| NA| null| null|
Reproducibility|2493746071; 21267...|Wenbo Li; Peiying...| 6| 4| 0| 0| NA| NA| NA| 0| 0| NA|accuracy and prec...|2017| Journal|Magnetic Resonanc...| Magn Reson Med| 10.1002/mrm.26325|Fast measurement ...| 5|Purpose
To develo...|WOS:000401270900020|MAGNETIC RESONANC...|
 OpenScience|2533450531; 21697...|Reham Al Tamime; ...| 3| 0| 2| 0.6666666666666666| 1|

In [9]:
openPapersJfinal.coalesce(1).write.format('com.databricks.spark.csv')\
.mode('overwrite').option("header", "true").save("/mnt/test/MAGwosJournalMatch/OpenSci3Journal.csv")